## Rozwiązywanie układów równań liniowych metodami interacyjnymi 

### Zadanie 1
Zaimplementuj metodę Jacobiego. Podaj warunki jej stosowalności. Wygeneruj co najmniej trzy odpowiednie macierze o różnych wielkościach i sprawdź działanie swojej metody. Zwróć uwagę na zbieżność tej metody. 

Warunkiem stosowalności metody Jacobiego jest aby macierz współczynników była nieredukowalna oraz diagonalnie dominująca. Z tego też powodu należy generować macierze spełniające powyższy warunek.

In [39]:
import numpy as np
import random


def jacobi(A, b, eps=1e-10, n=500):
    D = np.diag(np.diag(A))
    LU = A - D
    x = np.zeros(len(b))
    it = 0

    for i in range(n):
        D_inv = np.diag(1 / np.diag(D))
        x_new = np.dot(D_inv, b - np.dot(LU, x))
        it += 1
        if np.linalg.norm(x_new - x) < eps:
            return x_new, it
        x = x_new

    return x, it


def random_matrix_vector(m_min, m_max, v_min, v_max, n):
    A = np.zeros((n, n))
    b = np.random.uniform(low=v_min, high=v_max + 1, size=(n,))

    for i in range(n):
        sum = 0

        for j in range(n):
            if j >= i:
                A[i][j] = random.randint(m_min, m_max)
                A[j][i] = A[i][j]
            if i != j:
                sum += abs(A[i][j])

        A[i][i] = sum

    return A, b

def comparison(m_min, m_max, v_min, v_max, n_size, n_iter, eps=1e-10):
    A, b = random_matrix_vector(m_min, m_max, v_min, v_max, n_size)
    print(f"Numpy\n{np.linalg.solve(A, b)}")
    print(f"Jacobi method:\n{jacobi(A, b, eps, n_iter)}\n\n")

In [40]:
comparison(-15, 15, -10, 10, 5, 1000)

Numpy
[ 0.51616005  0.42450849 -0.3729423   0.06218176  0.01059072]
Jacobi method:
(array([ 0.51616005,  0.42450849, -0.3729423 ,  0.06218176,  0.01059072]), 188)




In [41]:
comparison(-25, 25, -5, 5, 5, 1000)

Numpy
[-0.04949928 -0.02824627  0.00558358  0.0125458   0.03091845]
Jacobi method:
(array([-0.04949928, -0.02824627,  0.00558358,  0.0125458 ,  0.03091845]), 722)




In [42]:
comparison(-15, 15, -20, 20, 5, 1000)

Numpy
[-5.2495625e+15 -5.2495625e+15  5.2495625e+15 -5.2495625e+15
  5.2495625e+15]
Jacobi method:
(array([ 410.31069052,  408.5978277 , -409.18965228,  409.11100193,
       -408.76051867]), 1000)




Jak widac powyżej, metoda Jacobiego działa zgodnie z oczekiwaniami.

### Zadanie 2
Zaimplementuj metodę Gaussa-Seidla i kolejnych nadrelaksacji (successive over-relaxation). Podaj warunki stosowalności tych metod. Przeprowadź badanie działania swoich implementacji analogicznie jak w poprzednim zadaniu. Porównaj zbieżność wszystkich trzech metod.

Metoda Gaussa-Seidela zachwouje warunki metody Jacobiego. Jeżeli macierz wejściowa jest dodatnio określona to metoda Gaussa-Seidela jest zbieżna dla dowolnego wektora początkowego [8, 10]. Metoda Gaussa-Seidela jest stosowana dla macierzy diagonalnie dominujących ponieważ zazwyczaj jest to łatwy do spełnienia warunek gwarantujący zbieżność metody.

Metoda kolejnych nadrelaksacji (SOR - Successive Over-Relaxation) jest to modyfikacja metody Gaussa-Seidela (warunki te same) również zchwouje warunki metody Gaussa. Przypsiesza ona zbieżność ciągu przez przemnożenie przez odpowiednio dobraną liczbę $\omega$. Parametr $\omega$ służy do polepszania (przyspieszania) zbiezności metody i może przyjmować wartości z przedziału (0, 2). Dla pozostałych wartości metoda może nie być zbieżna dla pewnych przybliżeń początkowych.

In [43]:
def gauss_seidel(A, b, n, eps):
    x = np.zeros_like(b)
    it = 0
    for _ in range(n):
        x_new = np.zeros_like(x)
        it += 1
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        if np.allclose(x, x_new, atol=eps, rtol=0):
            return x_new, it
        x = x_new
    return x, it


def sor(A, b, n, eps, omg):
    if omg < 0 or omg > 2:
        raise Exception("Omega must be from range (0, 2)")
    it = 0
    x = np.zeros_like(b)
    x_new = np.zeros_like(x)
    for _ in range(n):
        it += 1
        for i in range(b.shape[0]):
            old_sum = np.dot(A[i, i + 1:], x_new[i + 1:])
            new_sum = np.dot(A[i, :i], x[:i])
            x[i] = (b[i] - (old_sum + new_sum)) / A[i, i]
            x[i] = np.dot(x[i], omg) + np.dot(x_new[i], (1 - omg))
        if np.linalg.norm(np.dot(A, x) - b) < eps:
            return x_new, it
        x_new = x
    return x, it


def comparison2(m_min, m_max, v_min, v_max, n_size, n_iter, eps=1e-10, omg=1.5):
    A, b = random_matrix_vector(m_min, m_max, v_min, v_max, n_size)
    print(f"Numpy:\n{np.linalg.solve(A, b)}")
    print(f"Gauss_Seidel:\n{gauss_seidel(A, b, n_iter, eps)[0]}")
    print(f"SOR:\n{sor(A, b, n_iter, eps, omg)[0]}")

In [45]:
comparison2(-10, 10, -10, 10, 10, 1000)

Numpy:
[-0.12010686 -0.3120324   0.3806959   0.07000349 -0.10612008 -0.10246401
 -0.23998619 -0.19750562 -0.30412576  0.19027917]
Gauss_Seidel:
[-0.12010686 -0.3120324   0.3806959   0.07000349 -0.10612008 -0.10246401
 -0.23998619 -0.19750562 -0.30412576  0.19027917]
SOR:
[-0.12010686 -0.3120324   0.3806959   0.07000349 -0.10612008 -0.10246401
 -0.23998619 -0.19750562 -0.30412576  0.19027917]


In [46]:
comparison2(-20, 20, -20, 20, 20, 1000)

Numpy:
[ 0.09203289 -0.08881479 -0.07058771  0.09227802  0.11420467 -0.02042917
  0.0784208  -0.06383678 -0.00104755  0.05335582  0.11656632  0.12508545
  0.05302632  0.02733111 -0.02666288  0.00778972 -0.09718888  0.13375373
 -0.06777019 -0.04109474]
Gauss_Seidel:
[ 0.09203289 -0.08881479 -0.07058771  0.09227802  0.11420467 -0.02042917
  0.0784208  -0.06383678 -0.00104755  0.05335582  0.11656632  0.12508545
  0.05302632  0.02733111 -0.02666288  0.00778972 -0.09718888  0.13375373
 -0.06777019 -0.04109474]
SOR:
[ 0.09203289 -0.08881479 -0.07058771  0.09227802  0.11420467 -0.02042917
  0.0784208  -0.06383678 -0.00104755  0.05335582  0.11656632  0.12508545
  0.05302632  0.02733111 -0.02666288  0.00778972 -0.09718888  0.13375373
 -0.06777019 -0.04109474]


In [47]:
comparison2(-100, 100, -100, 100, 30, 1000)

Numpy:
[-0.0321854  -0.02216663  0.00018558  0.02486188 -0.02687494  0.07358538
 -0.02185133 -0.03050423  0.0257396  -0.02175729 -0.01278765 -0.026569
 -0.06761443 -0.02094063  0.06466263 -0.04195842  0.05118    -0.05362808
 -0.03105369  0.04592793 -0.00480286 -0.03075339 -0.06815752 -0.03933744
 -0.04156389  0.01124329 -0.02493045 -0.0113854  -0.05926526 -0.03378868]
Gauss_Seidel:
[-0.0321854  -0.02216663  0.00018558  0.02486188 -0.02687494  0.07358538
 -0.02185133 -0.03050423  0.0257396  -0.02175729 -0.01278765 -0.026569
 -0.06761443 -0.02094063  0.06466263 -0.04195842  0.05118    -0.05362808
 -0.03105369  0.04592793 -0.00480286 -0.03075339 -0.06815752 -0.03933744
 -0.04156389  0.01124329 -0.02493045 -0.0113854  -0.05926526 -0.03378868]
SOR:
[-0.0321854  -0.02216663  0.00018558  0.02486188 -0.02687494  0.07358538
 -0.02185133 -0.03050423  0.0257396  -0.02175729 -0.01278765 -0.026569
 -0.06761443 -0.02094063  0.06466263 -0.04195842  0.05118    -0.05362808
 -0.03105369  0.04592793 -0.0

Jak widać powyżej, wszystkie metody działają poprawnie.

In [48]:
def comparison3(m_min, m_max, v_min, v_max, n_size, n_iter, eps=1e-10, omg=1.5):
    A, b = random_matrix_vector(m_min, m_max, v_min, v_max, n_size)
    x, it = jacobi(A, b, eps, n_iter)
    print(f"Jacobi:\n Iterations: {it}\n{x}")
    x, it = gauss_seidel(A, b, n_iter, eps)
    print(f"Gauss_Seidel:\nIterations: {it}\n{x}")
    x, it = sor(A, b, n_iter, eps, omg)
    print(f"SOR:\nIterations: {it}\n{x}")

In [49]:
comparison3(-10, 10, -10, 10, 10, 1000)

Jacobi:
 Iterations: 39
[-0.0376315   0.18732407 -0.20419917 -0.18273092  0.21259866  0.02192202
  0.05981271  0.01540353 -0.14102215 -0.09272504]
Gauss_Seidel:
Iterations: 22
[-0.0376315   0.18732407 -0.20419917 -0.18273092  0.21259866  0.02192202
  0.05981271  0.01540353 -0.14102215 -0.09272504]
SOR:
Iterations: 24
[-0.0376315   0.18732407 -0.20419917 -0.18273092  0.21259866  0.02192202
  0.05981271  0.01540353 -0.14102215 -0.09272504]


In [50]:
comparison3(-20, 20, -20, 20, 20, 1000)

Jacobi:
 Iterations: 29
[ 0.08181247  0.05881358 -0.02398448  0.05757182  0.07543283  0.06127651
 -0.03311482 -0.01769118  0.0208985   0.03365785  0.13404553  0.10316285
 -0.12148313 -0.05605743  0.13003905 -0.01722555 -0.09508933  0.13136656
  0.07494945  0.08518107]
Gauss_Seidel:
Iterations: 13
[ 0.08181247  0.05881358 -0.02398448  0.05757182  0.07543283  0.06127651
 -0.03311482 -0.01769118  0.0208985   0.03365785  0.13404553  0.10316285
 -0.12148313 -0.05605743  0.13003905 -0.01722555 -0.09508933  0.13136656
  0.07494945  0.08518107]
SOR:
Iterations: 15
[ 0.08181247  0.05881358 -0.02398448  0.05757182  0.07543283  0.06127651
 -0.03311482 -0.01769118  0.0208985   0.03365785  0.13404553  0.10316285
 -0.12148313 -0.05605743  0.13003905 -0.01722555 -0.09508933  0.13136656
  0.07494945  0.08518107]


In [51]:
comparison3(-100, 100, -100, 100, 30, 1000)

Jacobi:
 Iterations: 22
[ 0.04988089  0.04765455  0.01221578  0.01301578  0.02232821 -0.0661494
  0.00239306  0.06815424  0.00018607  0.03658219  0.02321629 -0.05949013
 -0.01460656 -0.02366768 -0.06282118  0.00768406 -0.06449115 -0.06474164
  0.00571895 -0.05450752  0.01025136  0.03087156  0.00384137  0.04421521
  0.03588128  0.0188421  -0.00560743  0.06887877  0.06643239 -0.02254553]
Gauss_Seidel:
Iterations: 12
[ 0.04988089  0.04765455  0.01221578  0.01301578  0.02232821 -0.0661494
  0.00239306  0.06815424  0.00018607  0.03658219  0.02321629 -0.05949013
 -0.01460656 -0.02366768 -0.06282118  0.00768406 -0.06449115 -0.06474164
  0.00571895 -0.05450752  0.01025136  0.03087156  0.00384137  0.04421521
  0.03588128  0.0188421  -0.00560743  0.06887877  0.06643239 -0.02254553]
SOR:
Iterations: 16
[ 0.04988089  0.04765455  0.01221578  0.01301578  0.02232821 -0.0661494
  0.00239306  0.06815424  0.00018607  0.03658219  0.02321629 -0.05949013
 -0.01460656 -0.02366768 -0.06282118  0.00768406 -0.

Na podstawie powyższego porównania zbieżności można wywnioskować, że najszybciej zbieżna jest metoda Gaussa, następnie SOR a na końcu Jacobi